# Shamir

This script use the Shamir's sharing-secret schemes to encode a secret and decode a secret.

## How does it work ?

Shamir's sharing-secret scheme work with polynomials
A polynomial of n degree can be written like this

$f(x) = a_0 + a_2x + a_3x² + a_4x⁴+ \dots +a_nx^n$

This function can be represented in a 2 dimensional table represented with $x$ on vertical and $y$ or $f(x)$ on horizontal.
So a point has a coordinate $(x;y)$ or $(x;f(x))$

With the enough number of point, we can find the fonction $f$. This number correspond to the degree $n$ (Be careful, in the code, n and k don't correspond to these mathematical number)

So that's what Shamir used to create his secret-sharing scheme.
We first create a function $f$ with $f(0)$ equals the secret.
This is really easy as $f(0) = a_0$, so $a_0$ will be renamed $s$.
This function will have a degree that equals to the number of person that is needed $+1$


Then we give any points that is on the function creates (apart form 0) to people as this number is infinite.

To decode the secret we take the number of points that is needed to decode the secret an find the function $f$.

This program uses Gaussian pivots to find $f$ (but any methods works)
Here is the equation system that is needed to solve to find each coefficient and then $s$ the secret.
$$
\begin{cases} s+ax_1+bx_1²+cx_1³\dots = y_1\\ s+ax_2+bx_2²+cx_2³\dots = y_2\\ s+ax_3+bx_3²+cx_3³\dots = y_3\\s+ax_4+bx_4²+cx_4³\dots = y_4\\ \dots \end{cases}
$$

The detail of how the script solves this will be explained in *Gauss_Matrix.ipynb*.

### coder()

This function encode the secret

First it needs 3 arguments :

`` coder( n, k, s )``

$n$ is the number of share that we want to create.
 $k$ is the number of shares that is needed to find the secret.
$s$ is the secret.

First the algorithm create a function $f$ with $s$ as the first number than add random coefficient. ``k-1`` correspond to the degree of the polynomial.
```
import random
f= [secret]
for i in range(k-1):
    coef = 0
    while coef == 0:
        coef = random.randint(-30,30)
    f.append(coef)
```

Then it add random $x$ position and calculate $f(x)$.

```
for i in range(n):         
	while ptX in used_ptX:
	    ptX = random.randint(-n*3,n*3)
	    ptY = 0
	    degre = 0
	    for i in f:
	        ptY = ptY + i*(ptX**degre)
	        degre += 1
	used_ptX.append(ptX)
	pts.append("("+str(ptX)+";"+str(ptY)+")")

```
Last it add the coordinates to the *data.txt* file and print the different point

```
fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "w")
for i in pts:
    print(i)
    fichier.write(i+"\n")
```

In [ ]:
import random
def coder(n,k,secret): #avec k le nombre de points nécessaires pour décoder et n le nombre de points partagés
    if k <= 0:
        print("Erreur, cette valeur n'est pas valable")
    else :
        f = [secret]
        pts = []
        ptX = 0
        used_ptX = [0]
        for i in range(k-1):
            coef = 0
            while coef == 0:
                coef = random.randint(-30,30)
            f.append(coef)
        for i in range(n):
            
            while ptX in used_ptX:
                ptX = random.randint(-n*3,n*3)
                ptY = 0
                degre = 0
                for i in f:
                    ptY = ptY + i*(ptX**degre)
                    degre += 1
            
            used_ptX.append(ptX)
            pts.append("("+str(ptX)+";"+str(ptY)+")")
        print("Voici les différents points à distribuer à chaques personnes :")
        fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "w")
        for i in pts:
            print(i)
            fichier.write(i+"\n")


### decoder()

This function decode the secret.

First it needs 1 arguments :

`` decoder( k )``

$k$ is the number of shares that is needed to find the secret.

It first take only the points needed in *data.txt*
```
fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "r")
a = fichier.read()
fichier.close()
pts = []
coordonnes = []
for i in a :
    coordonnes.append(i)
v = 0
signe = 1
for i in coordonnes:
    if i in ["1","2","3","4","5","6","7","8","9","0"]:
        v = v*10 + int(i)
    elif i == "-":
        signe = -1
    elif i == ";" or i == ")":
        v = v * signe
        pts.append(v)
        signe = 1
        v = 0
```

Then it solve the system with the Matrix and print the secret.
The code will be explicated in *Gauss_Matrix.ipynb*.
```
if k <= 0:
    return "Erreur, une des valeurs n'est pas valable"
else:
    if len(pts)-2*k < 0:
        return "Les points fournis ne sont pas suffisants"
    for i in range(len(pts)-2*k):
        del pts[0]
    Matrice_Gauss = []
    for c in range(k):
        for l in range(k):
            Matrice_Gauss.append(pts[2*c]**l)
        Matrice_Gauss.append(pts[2*c+1])


    impossibilite = 0
    for colonneactive in range(k):
        
        pivot=Matrice_Gauss[colonneactive*(k+1)+colonneactive]
        if pivot == 0:
            impossibilite = 1
            break
        for l in range(k+1):
            
            Matrice_Gauss[colonneactive*(k+1)+l] = Matrice_Gauss[colonneactive*(k+1)+l]/pivot

        for c in range(k):
            coefcolonne = Matrice_Gauss[c*(k+1)+colonneactive]
            for l in range(k+1):
                if c != colonneactive:
                    Matrice_Gauss[c*(k+1)+l]=Matrice_Gauss[c*(k+1)+l]-Matrice_Gauss[colonneactive*(k+1)+l]*coefcolonne
    
    if impossibilite == 1:
        return "Les points fournis ne sont pas suffisants"
    else:
        print( "Avec les points :\n",a,"\n Le secret est : ",Matrice_Gauss[k], sep = "")

In [10]:
def decoder(k):
    fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "r")
    a = fichier.read()
    fichier.close()
    pts = []
    coordonnes = []
    for i in a :
        coordonnes.append(i)
    v = 0
    signe = 1
    for i in coordonnes:
        if i in ["1","2","3","4","5","6","7","8","9","0"]:
            v = v*10 + int(i)
        elif i == "-":
            signe = -1
        elif i == ";" or i == ")":
            v = v * signe
            pts.append(v)
            signe = 1
            v = 0
    


    if k <= 0:
        return "Erreur, une des valeurs n'est pas valable"
    else:
        if len(pts)-2*k < 0:
            return "Les points fournis ne sont pas suffisants"
        for i in range(len(pts)-2*k):
            del pts[0]
        Matrice_Gauss = []
        for c in range(k):
            for l in range(k):
                Matrice_Gauss.append(pts[2*c]**l)
            Matrice_Gauss.append(pts[2*c+1])


        impossibilite = 0
        for colonneactive in range(k):
            
            pivot=Matrice_Gauss[colonneactive*(k+1)+colonneactive]
            if pivot == 0:
                impossibilite = 1
                break
            for l in range(k+1):
                
                Matrice_Gauss[colonneactive*(k+1)+l] = Matrice_Gauss[colonneactive*(k+1)+l]/pivot

            for c in range(k):
                coefcolonne = Matrice_Gauss[c*(k+1)+colonneactive]
                for l in range(k+1):
                    if c != colonneactive:
                        Matrice_Gauss[c*(k+1)+l]=Matrice_Gauss[c*(k+1)+l]-Matrice_Gauss[colonneactive*(k+1)+l]*coefcolonne
        
        if impossibilite == 1:
            return "Les points fournis ne sont pas suffisants"
        else:
            print( "Avec les points :\n",a,"\n Le secret est : ",Matrice_Gauss[k], sep = "")

            
        


In [12]:
decoder(10)

Avec les points :
(-30;-287222936676307)
(9;6292908029)
(46;14070330303927109)
(-47;-16497734602787387)
(-41;-4813573127832221)
(-13;-148872962193)
(-5;-24230057)
(26;83890973263389)
(41;5005163674086429)
(10;16127024893)
(-26;-78882238445771)
(-33;-679004424298933)
(25;59007782100493)
(-2;-5147)
(-35;-1154772141817007)
(-48;-19946888227868263)
 Le secret est : -6.997314453125


In [ ]:
coder(5,3,8)
